In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.ndimage import uniform_filter1d, gaussian_filter1d
from toolkit.common.constants import *
from toolkit.lapsim.lap_sims import RunSim
from toolkit.cars.car_configuration import Car
from toolkit.lap.tracks import *

from toolkit.cars import sr_9
from toolkit.las_solvers import Octahedral_LAS, Multi_Layer_LAS, LAS
from toolkit.steady_state_solver import LS_Solver, Parachute, Iterative_Solver
from toolkit.tire_model.tire_model_utils import * # this has the tire models

In [ ]:
# import a track you want to run
track = get_MIS_2017_AX_3_track(50)
# track = get_Crows_2022(50)

In [ ]:
# import SR9 for instance
car = Car()
# car = sr_9()
car.set_tire(H_R20_18X6_7)
# solver = LS_Solver()
solver = Iterative_Solver()
# las = Octahedral_LAS(solver=solver)
las = Multi_Layer_LAS(solver=solver)
las.add_layers = 1
# car.debug = True

In [ ]:
# set some settings
# car.debug = True
v_average = 15
mu = 0.65 # mu correction factor, a value of 1 means the road is sandpaper and the actual value should be something lower but im kinda just setting this value to overfit atm
target = 0.0001

In [ ]:
# make the simulation object
sim = RunSim(track, car, las)

In [ ]:
# run the simulation
sim.simulate(sim_type='qts', v_average=v_average, mu=mu, convergence_target=target, bins=20)

In [ ]:
sim.plot_vs(distance=True, delta_beta_est=False, fz=False, separate_angles=True, yaw_acc=True, separate_acc=True, power_draw=False)

In [ ]:
fig = go.Figure()
find_zeros = np.where(sim.lon == 0)[0]
print(f"Number of zeros: {find_zeros}")

# plot the count of the number of times each point was solved
fig.add_trace(go.Scatter(x=np.arange(len(sim.count)), y=sim.count, mode='lines', marker=dict(size=2, color='red')))
fig.add_trace(go.Scatter(x=np.arange(len(sim.count)), y=sim.last_changed, mode='lines', marker=dict(size=2, color='blue')))
fig.add_trace(go.Scatter(x=find_zeros, y=np.zeros(len(find_zeros)), mode='markers', marker=dict(size=2, color='green')))
fig.update_layout(template="plotly_dark", title_text="Number of Points Solved")
fig.show()



In [ ]:
sim.lon

In [ ]:
# plot velocity and track curvature against the time gap
fig = make_subplots(rows=3, cols=2)
fig.add_trace(go.Scatter(x=sim.vel, y=sim.dt_dx, name='Velocity'), row=1, col=1)
fig.add_trace(go.Scatter(x=sim.track.k, y=np.interp(sim.track.u, sim.track.u_crit, sim.dt_dx), name='Curvature'), row=1, col=2)
fig.add_trace(go.Scatter(x=sim.lat, y=sim.dt_dx, name='Velocity'), row=2, col=1)
fig.add_trace(go.Scatter(x=sim.omega_dot, y=sim.dt_dx, name='Velocity'), row=2, col=2)
fig.add_trace(go.Scatter(x=sim.lon, y=sim.dt_dx, name='Velocity'), row=3, col=1)
fig.add_trace(go.Scatter(x=sim.omega, y=sim.dt_dx, name='Velocity'), row=3, col=2)
fig.update_layout(template="plotly_dark", title_text="Hi")
fig.show()
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=np.interp(sim.track.u_crit, sim.track.u, sim.track.K_prime)[2:], y=sim.vel[2:], mode='markers', marker={'color': np.abs(sim.dt_dx[2:])}, name='Velocity'))
fig2.update_layout(template="plotly_dark", title_text="Happy little christmas tree")
fig2.show()

In [ ]:
a, b, c, d, e = sim.track.interp_dist, sim.track.u_crit, sim.track.u, sim.track.spa, sim.track.ls_dist
fig = make_subplots(rows=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=c, y=-sim.track.real_beta, mode='lines', name="Beta Angle", legendgroup=f"group113", showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=b, y=sim.beta_est, mode='lines', name="Beta Angle Sim", legendgroup=f"group163", showlegend=True), row=1, col=1)
filtered_beta = gaussian_filter1d(sim.beta_est, 2.5)
fig.add_trace(go.Scatter(x=b, y=filtered_beta, mode='lines', name="Beta Angle Sim", legendgroup=f"group162", showlegend=True), row=1, col=1)
# fig.add_trace(go.Scatter(x=c, y=sim.track.k, mode='lines', name="Beta Angle Sim", legendgroup=f"group164", showlegend=True), row=1, col=1)
fig.show()
# abs average error
np.mean(np.abs(filtered_beta - sim.track.real_beta))
# as a percent of real
np.mean(np.abs(filtered_beta - sim.track.real_beta)) / np.mean(np.abs(sim.track.real_beta))
# rms error as a percent of real
# np.sqrt(np.mean((filtered_beta - sim.track.real_beta) ** 2)) / np.mean(sim.track.real_beta)

In [ ]:
from scipy.optimize import least_squares
vp = []
def cost(x, car, valid_points, ax_target, ay_target, yaw_target, v_avg):
    ay_it, cn_it, yaw_it, ax_it, bruh_it, _ = car.solve_for_long(v_avg, ax_target, delta_x=np.deg2rad(x[0]), beta_x=np.deg2rad(x[1]), use_drag=True, mu_corr=mu) # , ay_it=ay_target, zeros=False
    valid_points.append([x[0], x[1], ay_it * G, yaw_it, ax_it * G, v_avg])
    return np.array([abs(ay_it - ay_target), abs(yaw_it - yaw_target), abs(ax_it - ax_target)])
good_delta, good_beta = [], []
maxi = 300
beta_max = 25
delta_max = 15
goods = np.array([0, 0])
for i in range(0, maxi):
    # goods = np.array([sim.delta_est[i], sim.beta_est[i]])
    ax_t, ay_t, yaw_t, v_avg = sim.long[i] / G, sim.lat[i] / G, sim.omega_dot[i], sim.vel[i]
    sol = least_squares(cost, goods, args=(car, vp, ax_t, ay_t, yaw_t, v_avg,), bounds=((-delta_max, -beta_max), (delta_max, beta_max)), jac='2-point', verbose=0)
    # print(sol.x)
    # print(cost(sol.x, car, vp, ax_t, ay_t, yaw_t, v_avg))
    good_delta.append(sol.x[0])
    good_beta.append(sol.x[1])
    goods = np.array(sol.x)

from plotly.subplots import make_subplots
fig = make_subplots(rows=2, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=sim.track.u, y=sim.track.real_beta, mode='lines', name="Beta Angle", legendgroup=f"group113", showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=sim.track.u_crit[:maxi], y=good_beta, mode='lines', name="Beta Angle Sim", legendgroup=f"group163", showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=b, y=filtered_beta, mode='lines', name="Beta Angle Sim Filtered", legendgroup=f"group162", showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=sim.track.u_crit, y=sim.delta_est, mode='lines', name="Delta Angle Sim Est", legendgroup=f"group143", showlegend=True), row=2, col=1)
fig.add_trace(go.Scatter(x=sim.track.u_crit[:maxi], y=good_delta, mode='lines', name="Delta Angle Sim", legendgroup=f"group123", showlegend=True), row=2, col=1)
fig.show()

# plot delta and beta tested
valids = np.array(vp)
# fig = make_subplots(rows=1, shared_xaxes=True, vertical_spacing=0.02)
# fig.add_trace(go.Scatter(x=valids[:, 0], y=valids[:, 1], mode='markers', name="Beta Angle", legendgroup=f"group113", showlegend=True), row=1, col=1)
# fig.show()

# plot ay and yaw for tested points
fig = make_subplots(rows=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=valids[:, 2], y=valids[:, 3], mode='markers', name="Beta Angle", legendgroup=f"group113", showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=sim.lat[:maxi], y=sim.omega_dot[:maxi], mode='markers', name="Real", legendgroup=f"group163", showlegend=True), row=1, col=1)
fig.show()

In [ ]:
angle_corr = uniform_filter1d(sim.track.real_angle_raw - np.deg2rad(np.interp(sim.track.spa, sim.track.u, sim.track.angle)), 1000)
real_angle = sim.track.real_angle_raw - angle_corr # this should probably be a gaussian filter
real_beta = np.interp(sim.track.u, sim.track.spa, np.rad2deg(real_angle)) - sim.track.angle
fig = go.Figure()
fig.add_trace(go.Scatter(x=sim.track.u, y=real_beta, mode='lines', name="Beta Angle", showlegend=True))
fig.add_trace(go.Scatter(x=sim.track.spa, y=np.rad2deg(angle_corr), mode='lines', name="angle_corr", showlegend=True))
fig.add_trace(go.Scatter(x=sim.track.spa, y=np.rad2deg(real_angle), mode='lines', name="real_angle", showlegend=True))
fig.add_trace(go.Scatter(x=sim.track.spa, y=np.rad2deg(sim.track.real_angle_raw), mode='lines', name="real_angle_raw", showlegend=True))
fig.add_trace(go.Scatter(x=sim.track.u, y=sim.track.angle, mode='lines', name="angle", showlegend=True))
fig.show()

In [ ]:
sim.plot_mmd3d()

In [ ]:
sim.plot_gg_vel()

In [ ]:
sim.plot_mmd_vel()

In [ ]:
sim.map_plot()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=np.abs(sim.track.real_beta), color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=sim.omega, color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=sim.track.k, color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=sim.track.u, color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())
fig = px.scatter_mapbox(lat=sim.track.lat_ss, lon=sim.track.lon_ss, color=sim.track.angle, color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

In [ ]:
# plot the results, this plots them all together but you can also plot them individually
sim.plot()

In [ ]:
print(sim.car.vels)

In [ ]:
print(sim.car.vps)

In [ ]:
sim.plot_LAS_lims()

In [ ]:
sim.plot_convergence()